In [55]:
import numpy as np
from scipy.stats import norm
from scipy.stats import uniform
from scipy.stats import gamma
from scipy.stats import expon
from scipy.stats import binom
import matplotlib.pyplot as plt



Designing scoring rules sampling on gaussian distribution

In [30]:
#
data_normal = norm.rvs(size=1000, loc=0, scale = 1)

# Creating a couple potential distributions to compare with
normal1 = norm(loc=0, scale = 1)
normal2 = norm(loc=1, scale = 1)
normal3 = norm(loc=0, scale = 2)
binom1 = binom(n=2,p=0.5)


#print(data_normal)
print(normal1.rvs(10))

[-1.4265498   1.27965825  1.89078308 -0.66636748  0.72909906  1.65086434
 -0.47101462  1.21869074  1.1886383   2.94490197]


### Energy Score
$\textnormal{Energy Score} (F,x) = 2E_F[||X-x||] - E_F[||X-X'||]$

In [31]:
#First example of energy score is when distribution is constant
#1 dimensional data
def energyscore1(samples, predist):

    # sample to true 
    M=samples.size
    distsamples = predist.rvs(M)

    E1 = sum(abs(distsamples - samples))/M
    #pairs of samples to itself
    #print(distsamples - distsamples.reshape(-1,1))
    E2 = np.sum(abs(distsamples - distsamples.reshape(-1,1)))/(M*(M-1))  ##This line is bad storage wise

    score = 2*E1 - E2
    print(score)
    return(score)

In [32]:
## Testing on different distributions
data_normaltest = norm.rvs(size=1000, loc=1, scale = 2)

#
energyscore1(data_normaltest, normal1)
energyscore1(data_normaltest, normal2)
energyscore1(data_normaltest, normal3)
energyscore1(data_normaltest, binom1)

#distribution closest does have lowest energy score


2.7606140733487035
2.4228473886883832
2.5252476974745433
2.634777664467761


np.float64(2.634777664467761)

In [33]:
#Second example is when you have pairs of different distributions and samples
#Assuming # of samples is equivalent to number of distributions
def energyscore2(samples, dists):

    # sample to true 
    M=samples.size
    distsamples = norm.rvs(loc=dists[:,0], scale = dists[:,1])

    E1 = sum(abs(distsamples - samples))/M
    #pairs of samples to itself
    E2 = np.sum(abs(distsamples - distsamples.reshape(-1,1)))/(M*(M-1))  ##This line is bad storage wise

    score = 2*E1 - E2
    print(score)
    return(score)

In [34]:
data_normaltest = norm.rvs(size=1000, loc=1, scale = 2)

#Creating sequences of distributions (mean 0:1, scale 1:2) for guassian distributions.
locs = uniform.rvs(loc=0, scale=1, size =1000)
scales = uniform.rvs(loc =1, scale =1, size = 1000)
params1 = np.column_stack((locs,scales))


# creating sequences of distributions, (mean 0.5:1.5, scale 1:2)
locs = uniform.rvs(loc=0.5, scale=1, size =1000)
scales = uniform.rvs(loc =1, scale =1, size = 1000)
params2 = np.column_stack((locs,scales))


energyscore2(data_normaltest, params1)
energyscore2(data_normaltest, params2) #better most times as mean is closer




2.289224841605872
2.218217087923419


np.float64(2.218217087923419)

## SEEPS
Making example data 

In [49]:
#Okay forecasting, 50% dry correct, 100% light correct, 80% heavy correct
contingency1 = [[50,0,0],[50,50,10],[0,0,40]]
#Fully correct forecast for same observation
contingency2 = [[100,0,0],[0,50,0],[0,0,50]]

#probability of D, L, H 0.5, 0.25, 0.25

S = [[0,2,6],[2,0,4],[10/3,4/3,0]]

def SEEPS(cont, score):
    mult = np.multiply(cont, score)
    sums = mult.sum()

    print(sums)
    return(sums)


#SEEPS(contingency1,S)
#SEEPS(contingency2,S)

#Perfect forecast has SEEPS of 0
#Can see how scoring matrix weights difference between dry and heavy much more heavily. 

def SEEPSfull(cont):
    col_sums = np.sum(cont, axis = 0)
    total_events = col_sums.sum()
    pD = col_sums[0]/total_events
    pL = col_sums[1]/total_events #not used
    pH = col_sums[2]/total_events

    S = [[0,1/pD,1/pH + 1/(1-pD)],[1/pD,0,1/pH],[1/pD + 1/(1-pH), 1/(1-pH), 0]]

    mult = np.multiply(cont, S)
    sums = mult.sum()

    print(sums)
    return(sums)

SEEPSfull(contingency1)
SEEPSfull(contingency2)


    


140.0
0.0


np.float64(0.0)

### Variogram Scoring Rule

In [93]:
#Starting with 1 dimensional data
#1 dimensional normal not great for variograms distances.
data_normaltrue = norm.rvs(size=1000, loc=0, scale = 1)
data_normalpred = norm.rvs(size=1000, loc=0, scale = 1)




In [94]:
##Need to look at lag vs covariance 
#lag for distance or time spatial patterns
#also covariance for spatial covariance patterns


#lag 10?
def variogram_score(observed, predictions, lag=10):
    variogramobserved = compute_variogram(observed, observed, lag)
    variogrampredicted = compute_variogram(predictions, predictions, lag)
    score = np.sum((variogramobserved - variogrampredicted) ** 2)
    print(score)
    return score


def compute_variogram(observed, unused, lag):
    n = len(observed)
    variogram = []
    for h in range(1, lag + 1):
        totaldifs = 0
        count = 0
        for i in range(n - h):
            #half squared distance
            totaldifs += 0.5 * (observed[i] - observed[i + h]) ** 2
            count += 1
        # Average variogram for the given lag
        variogram.append(totaldifs / count)
    
    #print(variogram)
    return np.array(variogram)

compute_variogram(data_normaltrue,data_normaltrue, 10)

#As all just normal instances, changing the lag doesn't change 

array([0.97915165, 1.02788646, 0.951775  , 0.94451351, 0.91145641,
       0.95575054, 0.97301219, 1.01818892, 0.99139775, 0.95573447])

In [99]:
# Parameters
# Calculate the variogram score
score = variogram_score(data_normaltrue, data_normalpred, lag=10)
#closer to zero is better


0.19257717498170174


In [ ]:
## Using covariance
